In [5]:
import numpy as np
import time

def read_input_from_file(filename):
    with open(filename, 'r') as file:
        lines = file.readlines()
    N, D, A, B = map(int, lines[0].split())
    dayoff = np.zeros((N+1, D+1))
    for i in range(1, N+1):
        numbers = list(map(int, lines[i].split()))
        for j in range(0, len(numbers) - 1):
            dayoff[i][numbers[j]] = 1
    return N, D, A, B, dayoff

In [6]:
import numpy as np
import time

filename = 'Testcase/tc'
output = 'Output/output'

list_test = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
for tc in list_test:
    start_time = time.time()
    input_file = f"{filename}{tc}.txt"
    output_file = f"{output}{tc}.txt"

    
    N, D, A, B, dayoff = read_input_from_file(input_file)

    x = np.zeros((N+1, D+1))

    # Tính số ca đêm tối thiểu trung bình mỗi nhân viên
    num_of_night_shifts = A * D
    min_night_shifts = 0
    if num_of_night_shifts % N != 0:
        min_night_shifts = num_of_night_shifts // N + 1
    else:
        min_night_shifts = num_of_night_shifts // N

    max_night_per_staff = np.zeros((N+1))
    #print("min_night_shifts: ", min_night_shifts)

    # Phân bổ ca đêm (ca 4)
    for j in range(1, D+1):
        # Đếm số nhân viên có thể làm việc trong ngày j
        available_staff = []
        for i in range(1, N+1):
            if dayoff[i][j] == 0 and x[i][j-1] != 4 and x[i][j] == 0:  
                available_staff.append(i)
        

        # Kiểm tra tính khả thi
        if len(available_staff) < 4 * A:
            print(f"Không khả thi: Ngày {j} không đủ nhân viên ({len(available_staff)} < {4*A})")
            break

        # Phân bổ ca 4
        nums = 0
        available_staff.sort(key=lambda i: max_night_per_staff[i])
        for i in available_staff:
            if nums < A and max_night_per_staff[i] <= min_night_shifts:
                x[i][j] = 4
                nums += 1
                max_night_per_staff[i] += 1
            if nums == A:
                break

        # Nếu không đủ A nhân viên cho ca 4, thử phân bổ thêm
        if nums < A:
            remaining_staff = [i for i in available_staff if x[i][j] == 0]
            remaining_staff.sort(key=lambda i: max_night_per_staff[i])
            for i in remaining_staff:
                if x[i][j] == 0 and nums < A:
                    x[i][j] = 4
                    nums += 1
                    max_night_per_staff[i] += 1
                if nums == A:
                    break

        if nums < A:
            print(f"Vi phạm ràng buộc: Ngày {j} không đủ {A} nhân viên cho ca 4 (chỉ có {nums})")

    print("max_night_per_staff: ", np.max(max_night_per_staff[1:]))

    # Phân bổ ca ngày (ca 1, 2, 3)
    for j in range(1, D+1):
        # Lấy danh sách nhân viên có thể làm việc
        available_staff = []
        for i in range(1, N+1):
            if dayoff[i][j] == 0 and x[i][j] == 0 and x[i][j-1] != 4:  # Không nghỉ phép và chưa được gán ca 4
                available_staff.append(i)

        # Phân bổ ca 1, 2, 3
        for shift in range(1, 4):
            nums = 0
            for i in available_staff:
                if x[i][j] == 0 and nums < A:
                    x[i][j] = shift
                    nums += 1
                if nums == A:
                    break


            if nums < A:
                print(f"Vi phạm ràng buộc: Ngày {j}, ca {shift} không đủ {A} nhân viên (chỉ có {nums})")

    # Ghi kết quả ra file
    with open(output_file, 'w') as out_file:
        
        for i in range(1, N+1):
            out_file.write(' '.join([str(int(x)) for x in x[i][1:]]) + '\n')
        end_time = time.time()
        out_file.write("Execution time: {:.2f} seconds\n".format(end_time - start_time))

max_night_per_staff:  1.0
max_night_per_staff:  5.0
max_night_per_staff:  9.0
max_night_per_staff:  2.0
max_night_per_staff:  9.0
max_night_per_staff:  19.0
max_night_per_staff:  1.0
max_night_per_staff:  7.0
max_night_per_staff:  5.0
max_night_per_staff:  1.0
